In [4]:
import soundfile as sf #Libreria para el analisis de audio compatible con numpy
import numpy as np
import matplotlib.pyplot as plt
import os #Modulo para el uso de funciones del sistema operativo
import pandas as pd
from scipy.signal import find_peaks

## Importamos los aechivos de la carpeta
Para importar los archivos realizamos el siguiente proceso:
1- Obtenemos los nombres de los archivos de la carpeta.
2-. Obtenemos la ruta de cada uno de los archivos y las guardamos en un arreglo
3-. Con la ruta y u ciclo for, importamos cada uno de los archivos al script y realizamos la operacion.

In [15]:
#Obtenemos la lista de los nombres de la carpeta con los audios
ruta = r"C:\Users\usuario\Downloads\Proyecto\Phonemas"
Audios = os.listdir(ruta)
#Obtenemos las direcciones de los audios concatenando los nombres de los phonemas con la ruta de la carpeta
Phonemas = [os.path.join(ruta, archivo) for archivo in Audios]
#Utilizamos la funcion split para eliminar el formato.wav de los nombres
audios_sf = [nombre.split(".wav")[0] for nombre in Audios]
#print(Phonemas)
print(audios_sf)

#DEclaramos una matris con el tamaño de los fonemas
prim_arm = np.zeros(len(Phonemas))
seg_arm = np.zeros(len(Phonemas))
ter_arm = np.zeros(len(Phonemas))
print(len(prim_arm))

['A10_10', 'A10_12', 'A10_14', 'A10_15', 'A10_16', 'A11_12', 'A11_14', 'A11_16', 'A11_17', 'A12_1', 'A13_1', 'A13_10', 'A13_17', 'A13_18', 'A13_20', 'A13_3', 'A13_4', 'A13_5', 'A13_6', 'A13_9', 'A14_1', 'A14_17', 'A14_18', 'A14_2', 'A14_20', 'A14_3', 'A14_4', 'A14_5', 'A14_6', 'A14_9', 'A15_1', 'A15_15', 'A15_19', 'A15_2', 'A15_3', 'A15_5', 'A15_8', 'A1_13', 'A2_10', 'A2_8', 'A3_3', 'A3_6', 'A3_8', 'A4_5', 'A5_10', 'A5_13', 'A6_10', 'A6_13', 'A9_10', 'A9_12', 'A9_14', 'A9_15', 'A9_17', 'E10_12', 'E10_13', 'E10_14', 'E10_15', 'E10_16', 'E11_12', 'E11_14', 'E11_16', 'E11_17', 'E12_1', 'E13_1', 'E13_10', 'E13_17', 'E13_18', 'E13_2', 'E13_20', 'E13_3', 'E13_4', 'E13_5', 'E13_6', 'E13_9', 'E14_1', 'E14_17', 'E14_18', 'E14_2', 'E14_20', 'E14_3', 'E14_4', 'E14_5', 'E14_6', 'E14_9', 'E15_1', 'E15_15', 'E15_17', 'E15_19', 'E15_2', 'E15_3', 'E15_5', 'E15_9', 'E1_13', 'E2_10', 'E2_8', 'E3_3', 'E3_6', 'E3_8', 'E4_5', 'E5_10', 'E5_13', 'E6_10', 'E6_13', 'E7_7', 'E8_9', 'E9_10', 'E9_12', 'E9_14', 'E

Ya que tenemos las rutas guardadas en la variable Phonemas, podemos proceder importar cada uno de los audios de manera individual y aplicarle su correspondiente transformada de Fourier. Para ello, primero definimos los parametros iniciales y despues procedemos a los calculos.

In [16]:
for i in range(len(Phonemas)):
    #X es el valor de los datos rescatados del audio, Fs es la frecuencia de muestreo utilizada en cada audio
    x, Fs = sf.read(Phonemas[i]) 
    X = np.fft.fft(x)
    Spectre = np.fft.fftshift(np.abs(X))
    N = len(Spectre) 
    w = np.linspace(-Fs/2,Fs/2,N)
    plt.figure()
    plt.plot(w, Spectre)
    plt.savefig(os.path.join(r"C:\Users\usuario\Downloads\Proyecto\Transformadas_phonemas", f"TF_" + audios_sf[i]))
    plt.close()
    #Encontramos el indice del valor maximo del arreglo del espectro
    picos, _= find_peaks(Spectre, distance = 100)
    #print(picos)
    picos_ordenados = sorted(picos, key=lambda i: Spectre[i], reverse=True) #utilizamos sorted para organizar las frecuencias
    primero = picos_ordenados[1]
    Segundo = picos_ordenados[3]
    Tercero = picos_ordenados[5]

    prim_arm[i] = np.abs(w[primero]) #le pongo el valor absoluto porque algunos picos quedan en frecuencias negativas, para obtener el valor real
    seg_arm[i] = np.abs(w[Segundo])
    ter_arm[i] = np.abs(w[Tercero])

Para poder almacenar los datos usamos un dataframe, donde almacenamos el indice, el nombre del archivo y el 

In [17]:
idAud = pd.DataFrame(audios_sf,columns= ['No. Paciente'])
Primer_armonico = pd.DataFrame(prim_arm, columns= ['Frecuencia del primer armonico'])
Segundo_armonico = pd.DataFrame(seg_arm, columns= ['Frecuencia del segundo armonico'])
Tercer_armonico = pd.DataFrame(ter_arm, columns= ['Frecuencia tercer armonico'])

dataframe = pd.concat([idAud,Primer_armonico, Segundo_armonico, Tercer_armonico], axis = 1)
#print(dataframe)

Vocal = dataframe.groupby(dataframe["No. Paciente"].str[0])
dataframes_por_letra = []

#Revisar esta estructura.
for letra, grupo in Vocal:
    dataframes_por_letra.append(grupo)

Vocales = ['a', 'e', 'i', 'm', 'o', 'u']

#Guardamos cada uno de los datasets obtenidos en un excel de manera individual
for i in range (len(Vocales)):
    dataframes_por_letra[i].to_excel('Transformdas_{}.xlsx'.format(Vocales[i]))

